In [12]:
corpus = train_df['Content'].apply(lambda x: x.split()).tolist() # 切分词语
for document in corpus[:5]:
    print(document)

['China’s', 'disastrous', 'job', 'prospects', 'is', 'bringing', 'social', 'changes.']
['China’s', 'zero-Covid', 'policy', 'comes', 'under', 'fire']
['Well']
['Now', 'when', 'I', 'think', 'about', 'it...', 'what', 'if', "Ayato's", '""hair', 'changes""', 'were', 'due', 'to', 'him', '""looking', 'too', 'feminine""', 'and', 'China', 'censorship', 'brigade', 'was', 'like', '""NOPE']
['My', 'brothers', 'home', 'from', 'china', 'and', "he's", 'legit', 'a', 'different', 'person', 'lol.', 'grew', 'a', 'beard']


In [22]:
# 将文本转换为词嵌入向量
def text_to_vector(text):
    words = text.split()
    vector = np.zeros(model.vector_size)
    for word in words:
        if word in model.wv:
            vector += model.wv[word]
    vector /= len(words)  # 取平均值以防止一个句子过长
    return vector

X_train_vec = np.vstack([text_to_vector(text) for text in train_df['Content']])
X_predict_vec = np.vstack([text_to_vector(text) for text in predict_df['Content']])


In [ ]:
# 文本向量化：使用 Word2Vec 词嵌入
model = Word2Vec.load('word2vec_social.model')  
words = train_df['Content'].dropna().tolist()  # 删除空值
word_vectors = [model[w] for w in words if w in model]

# 将词嵌入转换为适合机器学习模型的格式
X_train = np.vstack([word_vectors[word] for word in words])
X_predict = np.vstack([word_vectors[word] for word in predict_df['Content'].dropna().tolist() if word in model])


##  xgboost

In [45]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
     -------------------------------------- 124.9/124.9 MB 1.1 MB/s eta 0:00:00




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: D:\anaconda3\python.exe -m pip install --upgrade pip


## 目前按照顺序跑了数据1的主题和数据2的情感，先未进行预处理用朴素贝叶斯分类器、随机森林、SVM，大致了解了数据，发现机器学习对’无‘这类的识别准确性很低，后有调整一定的参数（树的深度（max_depth）、最小样本分割（min_samples_split）等参数）、使用词袋（Bag of Words）或TF-IDF等方法将文本转换为数值型、利用词嵌入如Word2Vec捕获语义信息、使用现有的情感词典来为文本中的词汇分配情感权重，提高情感分析的准确率（上述调整方法都是正在进行的）

In [1]:
pip install transformers torch scikit-learn pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: D:\anaconda3\python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
from transformers import BertTokenizer, BertModel

# 加载本地的BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained(r'C:\Users\123\twitter')
bert_model = BertModel.from_pretrained(r'C:\Users\123\twitter')


D:\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
# 加载情感词典
lexicoder_dict = pd.read_csv(r'C:\Users\123\twitter\Estimated_semantic_dimensions_bert_English.csv')

# 创建情感词典，假设包含'word'和'Emotion'（你可以根据需要选择合适的列）
sentiment_dict = dict(zip(lexicoder_dict['word'], lexicoder_dict['Emotion']))

In [5]:


# 使用LabelEncoder编码Area标签
label_encoder = LabelEncoder()
data['Area_encoded'] = label_encoder.fit_transform(data['Area'])

# 提取文本和情感标签
texts = data['Content'].tolist()
labels = data['Area_encoded'].tolist()


In [6]:
# 提取BERT的特征（token embeddings）
def get_bert_embeddings(text):
    # 使用BERT tokenizer对文本进行编码
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    # 获取BERT模型的最后一层隐藏状态（token embeddings）
    embeddings = outputs.last_hidden_state
    # 返回[CLS] token的embedding（通常用于句子级别的特征）
    return embeddings[:, 0, :].squeeze().numpy()

# 提取所有文本的BERT特征
bert_embeddings = np.array([get_bert_embeddings(text) for text in texts])


In [15]:
# 获取所有的 Area 类别
areas = data['Area'].unique()

# 输出不重复的值
print(areas)

In [7]:
# 将情感词典权重与BERT特征结合
def apply_sentiment_weights(bert_feature, text):
    # 使用CountVectorizer提取词频
    vectorizer = CountVectorizer(vocabulary=tokenizer.get_vocab())
    word_counts = vectorizer.fit_transform([text]).toarray().flatten()
    
    # 获取所有词汇
    words = vectorizer.get_feature_names_out()
    
    # 计算权重，词汇出现时，应用情感词典的权重
    weighted_bert_feature = bert_feature.copy()
    for idx, word in enumerate(words):
        if word in sentiment_dict:
            sentiment_weight = sentiment_dict[word]
            weighted_bert_feature += sentiment_weight * word_counts[idx]
    
    return weighted_bert_feature

# 加权BERT特征
weighted_bert_features = np.array([apply_sentiment_weights(bert_embeddings[i], texts[i]) for i in range(len(texts))])
 

D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [9]:
# 获取所有的 Area 类别
areas = data['Area'].unique()

# 遍历每个 Area 类别进行训练与预测
for area in areas:
    print(f"\nTraining and predicting for area: {area}")
    
    # 筛选出当前 Area 的数据
    area_data = data[data['Area'] == area]

    # 根据 train_predict 列将数据分为训练集和预测集
    train_data = area_data[area_data['train_predict'] == 'train']
    predict_data = area_data[area_data['train_predict'] == 'predict']

    # 检查并处理空值
    train_data = train_data.dropna(subset=['Content'])
    predict_data = predict_data.dropna(subset=['Content'])
    
    # 预测集的真实标签（y_true）
    label_encoder = LabelEncoder()
    y_true_emotion = label_encoder.fit_transform(predict_data['信息的对华情感'])  # Fit to the prediction set
    y_true_private_emotion = label_encoder.transform(predict_data['信息的私人情感（不指向CHN）'])  # Use transform for prediction labels

    # 检查训练集是否包含至少两个类别
    if len(train_data['信息的对华情感'].unique()) > 1 and len(train_data['信息的私人情感（不指向CHN）'].unique()) > 1:
        
        # 标签编码
        y_train_emotion = label_encoder.fit_transform(train_data['信息的对华情感'])  # Fit on train data
        y_train_private_emotion = label_encoder.fit_transform(train_data['信息的私人情感（不指向CHN）'])  # Fit on train data

        # 使用Logistic Regression模型进行训练
        model_emotion = LogisticRegression(max_iter=1000)
        model_emotion.fit(weighted_bert_features, y_train_emotion)

        model_private_emotion = LogisticRegression(max_iter=1000)
        model_private_emotion.fit(weighted_bert_features, y_train_private_emotion)

        # 获取预测数据的BERT特征
        bert_embeddings_predict = np.array([get_bert_embeddings(text) for text in predict_data['Content'].astype(str)])
        X_predict_vec = bert_embeddings_predict

        # 使用情感词典权重与BERT特征结合
        weighted_bert_features_predict = np.array([apply_sentiment_weights(X_predict_vec[i], predict_data['Content'].iloc[i]) for i in range(len(X_predict_vec))])

        # 预测
        y_pred_emotion = model_emotion.predict(weighted_bert_features_predict)
        y_pred_private_emotion = model_private_emotion.predict(weighted_bert_features_predict)

        # 计算评估指标
        print("Accuracy for 信息的私人情感（不指向CHN）:", accuracy_score(y_true_private_emotion, y_pred_private_emotion))
        print("Classification Report for 信息的私人情感（不指向CHN）:\n", classification_report(y_true_private_emotion, y_pred_private_emotion))
        
        print("Accuracy for 信息的对华情感:", accuracy_score(y_true_emotion, y_pred_emotion))
        print("Classification Report for 信息的对华情感:\n", classification_report(y_true_emotion, y_pred_emotion))

        # 保存预测结果到新的 CSV 文件
        predict_data['Predicted_Emotion'] = label_encoder.inverse_transform(y_pred_emotion)
        predict_data['Predicted_Private_Emotion'] = label_encoder.inverse_transform(y_pred_private_emotion)
        predict_data.to_csv(r'E:\研一\计算传播\给学生1\第二步情感预测\{}_标签的数据预测结果.csv'.format(area), index=False)

        # 打印预测集的前十个结果
        print(f"前十个预测结果 for area {area}:")
        print(predict_data[['Content', '信息的对华情感','信息的私人情感（不指向CHN）', 'Predicted_Emotion', 'Predicted_Private_Emotion']].head(10))

        print(f"预测结果已成功保存到 CSV 文件 for {area}")
    else:
        print(f"Skipping training for area '{area}' due to insufficient class diversity in the training set.")



Training and predicting for area:  军事


D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.

Accuracy for 信息的私人情感（不指向CHN）: 0.45454545454545453
Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.25      0.33      0.29         6
           1       0.64      0.70      0.67        10
           2       0.33      0.17      0.22         6

    accuracy                           0.45        22
   macro avg       0.41      0.40      0.39        22
weighted avg       0.45      0.45      0.44        22

Accuracy for 信息的对华情感: 0.45454545454545453
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.73      0.67      0.70        12
           2       0.33      0.29      0.31         7

    accuracy                           0.45        22
   macro avg       0.35      0.32      0.33        22
weighted avg       0.50      0.45      0.48        22

前十个预测结果 for area  军事:
                                            

D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.

Accuracy for 信息的私人情感（不指向CHN）: 0.5357142857142857
Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.25      0.11      0.15         9
           1       0.50      0.33      0.40         3
           2       0.59      0.81      0.68        16

    accuracy                           0.54        28
   macro avg       0.45      0.42      0.41        28
weighted avg       0.47      0.54      0.48        28

Accuracy for 信息的对华情感: 0.6785714285714286
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         6
           2       0.73      0.95      0.83        20

    accuracy                           0.68        28
   macro avg       0.24      0.32      0.28        28
weighted avg       0.52      0.68      0.59        28

前十个预测结果 for area  体育:
                                              

...

D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.

Accuracy for 信息的私人情感（不指向CHN）: 0.5384615384615384
Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.50      0.65      0.57        23
           1       0.56      0.41      0.47        22
           2       0.67      0.57      0.62         7

    accuracy                           0.54        52
   macro avg       0.58      0.54      0.55        52
weighted avg       0.55      0.54      0.53        52

Accuracy for 信息的对华情感: 0.5
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.36      0.73      0.48        11
           1       0.71      0.59      0.64        29
           2       0.17      0.08      0.11        12

    accuracy                           0.50        52
   macro avg       0.41      0.47      0.41        52
weighted avg       0.51      0.50      0.49        52

前十个预测结果 for area  政治:
                                               Content 信息的对华情

D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.

Accuracy for 信息的私人情感（不指向CHN）: 0.2777777777777778
Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.60      0.50      0.55         6
           1       0.67      0.17      0.27        12
           2       0.00      0.00      0.00         0

    accuracy                           0.28        18
   macro avg       0.42      0.22      0.27        18
weighted avg       0.64      0.28      0.36        18

Accuracy for 信息的对华情感: 0.16666666666666666
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.50      0.67      0.57         3
           1       1.00      0.07      0.12        15
           2       0.00      0.00      0.00         0

    accuracy                           0.17        18
   macro avg       0.50      0.24      0.23        18
weighted avg       0.92      0.17      0.20        18

前十个预测结果 for area  文化:
                                             

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Reca

#  RandomForest

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# 获取所有的 Area 类别
areas = data['Area'].unique()

# 用于保存所有类别的预测结果
all_predictions = []

# 遍历每个 Area 类别进行训练与预测
for area in areas:
    print(f"\nTraining and predicting for area: {area}")
    
    # 筛选出当前 Area 的数据
    area_data = data[data['Area'] == area]

    # 根据 train_predict 列将数据分为训练集和预测集
    train_data = area_data[area_data['train_predict'] == 'train']
    predict_data = area_data[area_data['train_predict'] == 'predict']

    # 检查并处理空值
    train_data = train_data.dropna(subset=['Content'])
    predict_data = predict_data.dropna(subset=['Content'])
    
    # 预测集的真实标签（y_true）
    label_encoder = LabelEncoder()
    y_true_emotion = label_encoder.fit_transform(predict_data['信息的对华情感'])  # Fit to the prediction set
    y_true_private_emotion = label_encoder.transform(predict_data['信息的私人情感（不指向CHN）'])  # Use transform for prediction labels

    # 获取BERT特征
    bert_embeddings = np.array([get_bert_embeddings(text) for text in train_data['Content'].astype(str)])
    X_train = bert_embeddings
    
    # 使用情感词典权重与BERT特征结合
    weighted_bert_features = np.array([apply_sentiment_weights(X_train[i], train_data['Content'].iloc[i]) for i in range(len(X_train))])

    # 检查训练集是否包含至少两个类别
    if len(train_data['信息的对华情感'].unique()) > 1 and len(train_data['信息的私人情感（不指向CHN）'].unique()) > 1:
        
        # 标签编码
        y_train_emotion = label_encoder.fit_transform(train_data['信息的对华情感'])  # Fit on train data
        y_train_private_emotion = label_encoder.fit_transform(train_data['信息的私人情感（不指向CHN）'])  # Fit on train data

        # 使用Random Forest模型进行训练
        model_emotion = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
        model_emotion.fit(weighted_bert_features, y_train_emotion)

        model_private_emotion = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
        model_private_emotion.fit(weighted_bert_features, y_train_private_emotion)

        # 获取预测数据的BERT特征
        bert_embeddings_predict = np.array([get_bert_embeddings(text) for text in predict_data['Content'].astype(str)])
        X_predict_vec = bert_embeddings_predict

        # 使用情感词典权重与BERT特征结合
        weighted_bert_features_predict = np.array([apply_sentiment_weights(X_predict_vec[i], predict_data['Content'].iloc[i]) for i in range(len(X_predict_vec))])

        # 预测
        y_pred_emotion = model_emotion.predict(weighted_bert_features_predict)
        y_pred_private_emotion = model_private_emotion.predict(weighted_bert_features_predict)

        # 计算评估指标
        print("Accuracy for 信息的私人情感（不指向CHN）:", accuracy_score(y_true_private_emotion, y_pred_private_emotion))
        print("Classification Report for 信息的私人情感（不指向CHN）:\n", classification_report(y_true_private_emotion, y_pred_private_emotion))
        
        print("Accuracy for 信息的对华情感:", accuracy_score(y_true_emotion, y_pred_emotion))
        print("Classification Report for 信息的对华情感:\n", classification_report(y_true_emotion, y_pred_emotion))

        # 保存预测结果到数据集合并
        predict_data['Predicted_Emotion'] = label_encoder.inverse_transform(y_pred_emotion)
        predict_data['Predicted_Private_Emotion'] = label_encoder.inverse_transform(y_pred_private_emotion)
        all_predictions.append(predict_data)

        # 打印预测集的前十个结果
        print(f"前十个预测结果 for area {area}:")
        print(predict_data[['Content', '信息的对华情感','信息的私人情感（不指向CHN）', 'Predicted_Emotion', 'Predicted_Private_Emotion']].head(10))

    else:
        print(f"Skipping training for area '{area}' due to insufficient class diversity in the training set.")

# 合并所有类别的预测结果
final_predictions = pd.concat(all_predictions, axis=0)

# 保存所有预测结果到一个CSV文件
final_predictions.to_csv(r'E:\研一\计算传播\给学生1\第二步情感预测\random_forest_data_predictions.csv', index=False)

print(f"所有预测结果已成功保存到 CSV 文件.")



Training and predicting for area:  军事


D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


Accuracy for 信息的私人情感（不指向CHN）: 0.36363636363636365
Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.12      0.17      0.14         6
           1       0.50      0.70      0.58        10
           2       0.00      0.00      0.00         6

    accuracy                           0.36        22
   macro avg       0.21      0.29      0.24        22
weighted avg       0.26      0.36      0.30        22

Accuracy for 信息的对华情感: 0.5
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.55      0.92      0.69        12
           2       0.00      0.00      0.00         7

    accuracy                           0.50        22
   macro avg       0.18      0.31      0.23        22
weighted avg       0.30      0.50      0.38        22

前十个预测结果 for area  军事:
                                              Content 信息的对华情

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: Unde

Accuracy for 信息的私人情感（不指向CHN）: 0.5
Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.17      0.11      0.13         9
           1       1.00      0.33      0.50         3
           2       0.57      0.75      0.65        16

    accuracy                           0.50        28
   macro avg       0.58      0.40      0.43        28
weighted avg       0.49      0.50      0.47        28

Accuracy for 信息的对华情感: 0.6785714285714286
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       1.00      0.17      0.29         6
           2       0.72      0.90      0.80        20

    accuracy                           0.68        28
   macro avg       0.57      0.36      0.36        28
weighted avg       0.73      0.68      0.63        28

前十个预测结果 for area  体育:
                                               Content 信息的对华情

...

D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


Accuracy for 信息的私人情感（不指向CHN）: 0.4423076923076923
Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.44      0.78      0.56        23
           1       0.45      0.23      0.30        22
           2       0.00      0.00      0.00         7

    accuracy                           0.44        52
   macro avg       0.30      0.34      0.29        52
weighted avg       0.39      0.44      0.38        52

Accuracy for 信息的对华情感: 0.5
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.33      0.55      0.41        11
           1       0.65      0.69      0.67        29
           2       0.00      0.00      0.00        12

    accuracy                           0.50        52
   macro avg       0.33      0.41      0.36        52
weighted avg       0.43      0.50      0.46        52

前十个预测结果 for area  政治:
                                               Content 信息的对华情

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1381: User

Accuracy for 信息的私人情感（不指向CHN）: 0.16666666666666666
Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.25      0.17      0.20         6
           1       0.50      0.17      0.25        12
           2       0.00      0.00      0.00         0

    accuracy                           0.17        18
   macro avg       0.25      0.11      0.15        18
weighted avg       0.42      0.17      0.23        18

Accuracy for 信息的对华情感: 0.2777777777777778
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.50      0.67      0.57         3
           1       0.75      0.20      0.32        15
           2       0.00      0.00      0.00         0

    accuracy                           0.28        18
   macro avg       0.42      0.29      0.30        18
weighted avg       0.71      0.28      0.36        18

前十个预测结果 for area  文化:
                                             

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Reca

# xgboost

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

# 获取所有的 Area 类别
areas = data['Area'].unique()

# 用于保存所有类别的预测结果
all_predictions = []

# 遍历每个 Area 类别进行训练与预测
for area in areas:
    print(f"\nTraining and predicting for area: {area}")
    
    # 筛选出当前 Area 的数据
    area_data = data[data['Area'] == area]

    # 根据 train_predict 列将数据分为训练集和预测集
    train_data = area_data[area_data['train_predict'] == 'train']
    predict_data = area_data[area_data['train_predict'] == 'predict']

    # 检查并处理空值
    train_data = train_data.dropna(subset=['Content'])
    predict_data = predict_data.dropna(subset=['Content'])
    
    # 预测集的真实标签（y_true）
    label_encoder = LabelEncoder()
    y_true_emotion = label_encoder.fit_transform(predict_data['信息的对华情感'])  # Fit to the prediction set
    y_true_private_emotion = label_encoder.transform(predict_data['信息的私人情感（不指向CHN）'])  # Use transform for prediction labels

    # 获取BERT特征
    bert_embeddings = np.array([get_bert_embeddings(text) for text in train_data['Content'].astype(str)])
    X_train = bert_embeddings
    
    # 使用情感词典权重与BERT特征结合
    weighted_bert_features = np.array([apply_sentiment_weights(X_train[i], train_data['Content'].iloc[i]) for i in range(len(X_train))])

    # Ensure that the features are in 2D format for the model
    weighted_bert_features = weighted_bert_features.reshape(len(weighted_bert_features), -1)  # Reshape to 2D (n_samples, n_features)

    # 检查训练集是否包含至少两个类别
    if len(train_data['信息的对华情感'].unique()) > 1 and len(train_data['信息的私人情感（不指向CHN）'].unique()) > 1:
        
        # 标签编码
        y_train_emotion = label_encoder.fit_transform(train_data['信息的对华情感'])  # Fit on train data
        y_train_private_emotion = label_encoder.fit_transform(train_data['信息的私人情感（不指向CHN）'])  # Fit on train data

        # 使用XGBoost模型进行训练
        model_emotion = XGBClassifier(
            use_label_encoder=False,
            eval_metric='mlogloss',  # 多类对数损失
            learning_rate=0.1,  # 适度的学习率
            max_depth=3,  # 最大深度调整为3
            n_estimators=50,  # 树的数量设置为50
            subsample=0.8,  # 使用80%的样本进行训练
            colsample_bytree=0.8  # 每棵树使用80%的特征
        )
        model_emotion.fit(weighted_bert_features, y_train_emotion)

        model_private_emotion = XGBClassifier(
            use_label_encoder=False,
            eval_metric='mlogloss',
            learning_rate=0.1,
            max_depth=3,
            n_estimators=50,
            subsample=0.8,
            colsample_bytree=0.8
        )
        model_private_emotion.fit(weighted_bert_features, y_train_private_emotion)

        # 获取预测数据的BERT特征
        bert_embeddings_predict = np.array([get_bert_embeddings(text) for text in predict_data['Content'].astype(str)])
        X_predict_vec = bert_embeddings_predict

        # 使用情感词典权重与BERT特征结合
        weighted_bert_features_predict = np.array([apply_sentiment_weights(X_predict_vec[i], predict_data['Content'].iloc[i]) for i in range(len(X_predict_vec))])

        # Ensure that the features are in 2D format for the model
        weighted_bert_features_predict = weighted_bert_features_predict.reshape(len(weighted_bert_features_predict), -1)  # Reshape to 2D (n_samples, n_features)

        # 预测
        y_pred_emotion = model_emotion.predict(weighted_bert_features_predict)
        y_pred_private_emotion = model_private_emotion.predict(weighted_bert_features_predict)

        # 计算评估指标
        print("Accuracy for 信息的私人情感（不指向CHN）:", accuracy_score(y_true_private_emotion, y_pred_private_emotion))
        print("Classification Report for 信息的私人情感（不指向CHN）:\n", classification_report(y_true_private_emotion, y_pred_private_emotion))
        
        print("Accuracy for 信息的对华情感:", accuracy_score(y_true_emotion, y_pred_emotion))
        print("Classification Report for 信息的对华情感:\n", classification_report(y_true_emotion, y_pred_emotion))

        # 保存预测结果到数据集合并
        predict_data['Predicted_Emotion'] = label_encoder.inverse_transform(y_pred_emotion)
        predict_data['Predicted_Private_Emotion'] = label_encoder.inverse_transform(y_pred_private_emotion)
        all_predictions.append(predict_data)

        # 打印预测集的前十个结果
        print(f"前十个预测结果 for area {area}:")
        print(predict_data[['Content', '信息的对华情感','信息的私人情感（不指向CHN）', 'Predicted_Emotion', 'Predicted_Private_Emotion']].head(10))

    else:
        print(f"Skipping training for area '{area}' due to insufficient class diversity in the training set.")

# 合并所有类别的预测结果
final_predictions = pd.concat(all_predictions, axis=0)

# 保存所有预测结果到一个CSV文件
final_predictions.to_csv(r'E:\研一\计算传播\给学生1\第二步情感预测\xgboost_data_predictions.csv', index=False)

print(f"所有预测结果已成功保存到 CSV 文件.")



Training and predicting for area:  军事
Accuracy for 信息的私人情感（不指向CHN）: 0.45454545454545453


D:\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [01:09:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: Undef

Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.45      1.00      0.62        10
           2       0.00      0.00      0.00         6

    accuracy                           0.45        22
   macro avg       0.15      0.33      0.21        22
weighted avg       0.21      0.45      0.28        22

Accuracy for 信息的对华情感: 0.5454545454545454
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.55      1.00      0.71        12
           2       0.00      0.00      0.00         7

    accuracy                           0.55        22
   macro avg       0.18      0.33      0.24        22
weighted avg       0.30      0.55      0.39        22

前十个预测结果 for area  军事:
                                              Content 信息的对华情感 信息的私人情感（不指向CHN）  \
9             

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [01:09:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: Undef

Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.00      0.00      0.00         3
           2       0.57      1.00      0.73        16

    accuracy                           0.57        28
   macro avg       0.19      0.33      0.24        28
weighted avg       0.33      0.57      0.42        28

Accuracy for 信息的对华情感: 0.7142857142857143
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         6
           2       0.71      1.00      0.83        20

    accuracy                           0.71        28
   macro avg       0.24      0.33      0.28        28
weighted avg       0.51      0.71      0.60        28

前十个预测结果 for area  体育:
                                               Content 信息的对华情感  \
121                          

...

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: Unde

Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.50      1.00      0.67        21
           2       0.00      0.00      0.00         3

    accuracy                           0.50        42
   macro avg       0.17      0.33      0.22        42
weighted avg       0.25      0.50      0.33        42

Accuracy for 信息的对华情感: 0.5
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67        21
           2       0.00      0.00      0.00        13

    accuracy                           0.50        42
   macro avg       0.17      0.33      0.22        42
weighted avg       0.25      0.50      0.33        42

前十个预测结果 for area  社会:
                                               Content 信息的对华情感  \
427  China's cyber space regulator has sent 

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: Unde

Classification Report for 信息的私人情感（不指向CHN）:
               precision    recall  f1-score   support

           0       0.44      1.00      0.61        23
           1       0.00      0.00      0.00        22
           2       0.00      0.00      0.00         7

    accuracy                           0.44        52
   macro avg       0.15      0.33      0.20        52
weighted avg       0.20      0.44      0.27        52

Accuracy for 信息的对华情感: 0.5576923076923077
Classification Report for 信息的对华情感:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.56      1.00      0.72        29
           2       0.00      0.00      0.00        12

    accuracy                           0.56        52
   macro avg       0.19      0.33      0.24        52
weighted avg       0.31      0.56      0.40        52

前十个预测结果 for area  政治:
                                               Content 信息的对华情感  \
864                          

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMet

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer

# 获取所有的 Area 类别
areas = data['Area'].unique()

all_predictions_svm = []

# 遍历每个 Area 类别进行训练与预测
for area in areas:
    print(f"\nTraining and predicting for area: {area}")
    
    # 筛选出当前 Area 的数据
    area_data = data[data['Area'] == area]

    # 根据 train_predict 列将数据分为训练集和预测集
    train_data = area_data[area_data['train_predict'] == 'train']
    predict_data = area_data[area_data['train_predict'] == 'predict']

    # 检查并处理空值
    train_data = train_data.dropna(subset=['Content'])
    predict_data = predict_data.dropna(subset=['Content'])
    
    # 删除训练集和预测集中的包含NaN的行
    train_data = train_data.dropna()
    predict_data = predict_data.dropna()
    
    # 预测集的真实标签（y_true）
    label_encoder = LabelEncoder()
    y_true_emotion = label_encoder.fit_transform(predict_data['信息的对华情感'])  # Fit to the prediction set
    y_true_private_emotion = label_encoder.transform(predict_data['信息的私人情感（不指向CHN）'])  # Use transform for prediction labels

    # 获取BERT特征
    bert_embeddings = np.array([get_bert_embeddings(text) for text in train_data['Content'].astype(str)])
    X_train = bert_embeddings
    
    # 使用情感词典权重与BERT特征结合
    weighted_bert_features = np.array([apply_sentiment_weights(X_train[i], train_data['Content'].iloc[i]) for i in range(len(X_train))])

    # Ensure that the features are in 2D format for the model
    weighted_bert_features = weighted_bert_features.reshape(len(weighted_bert_features), -1)  # Reshape to 2D (n_samples, n_features)
    weighted_bert_features_predict = weighted_bert_features_predict.reshape(len(weighted_bert_features_predict), -1)

    # 检查训练集是否包含至少两个类别
    if len(train_data['信息的对华情感'].unique()) > 1 and len(train_data['信息的私人情感（不指向CHN）'].unique()) > 1:
        
        # 标签编码
        y_train_emotion = label_encoder.fit_transform(train_data['信息的对华情感'])  # Fit on train data
        y_train_private_emotion = label_encoder.fit_transform(train_data['信息的私人情感（不指向CHN）'])  # Fit on train data

        # 使用支持向量机（SVM）进行训练
        model_emotion_svm = SVC(kernel='linear', C=1.0)
        model_emotion_svm.fit(weighted_bert_features, y_train_emotion)

        model_private_emotion_svm = SVC(kernel='linear', C=1.0)
        model_private_emotion_svm.fit(weighted_bert_features, y_train_private_emotion)

        # 获取预测数据的BERT特征
        bert_embeddings_predict = np.array([get_bert_embeddings(text) for text in predict_data['Content'].astype(str)])
        X_predict_vec = bert_embeddings_predict

        # 使用情感词典权重与BERT特征结合
        weighted_bert_features_predict = np.array([apply_sentiment_weights(X_predict_vec[i], predict_data['Content'].iloc[i]) for i in range(len(X_predict_vec))])

        # 用均值填充预测集中的NaN值
        weighted_bert_features_predict = imputer.transform(weighted_bert_features_predict)

        # 预测（支持向量机）
        y_pred_emotion_svm = model_emotion_svm.predict(weighted_bert_features_predict)
        y_pred_private_emotion_svm = model_private_emotion_svm.predict(weighted_bert_features_predict)

        # 计算评估指标（支持向量机）
        print("SVM - Accuracy for 信息的私人情感（不指向CHN）:", accuracy_score(y_true_private_emotion, y_pred_private_emotion_svm))
        print("SVM - Classification Report for 信息的私人情感（不指向CHN）:\n", classification_report(y_true_private_emotion, y_pred_private_emotion_svm))

        print("SVM - Accuracy for 信息的对华情感:", accuracy_score(y_true_emotion, y_pred_emotion_svm))
        print("SVM - Classification Report for 信息的对华情感:\n", classification_report(y_true_emotion, y_pred_emotion_svm))

        # 保存预测结果到数据集合并
        predict_data['Predicted_Emotion_SVM'] = label_encoder.inverse_transform(y_pred_emotion_svm)
        predict_data['Predicted_Private_Emotion_SVM'] = label_encoder.inverse_transform(y_pred_private_emotion_svm)

        all_predictions_svm.append(predict_data[['Content', 'Predicted_Emotion_SVM', 'Predicted_Private_Emotion_SVM']])

        # 打印预测集的前十个结果
        print(f"前十个预测结果 for area {area}:")
        print(predict_data[['Content', '信息的对华情感', '信息的私人情感（不指向CHN）', 
                            'Predicted_Emotion_SVM', 'Predicted_Private_Emotion_SVM']].head(10))

    else:
        print(f"Skipping training for area '{area}' due to insufficient class diversity in the training set.")

# 合并所有类别的预测结果（支持向量机）
final_predictions_svm = pd.concat(all_predictions_svm, axis=0)
final_predictions_svm.to_csv(r'E:\研一\计算传播\给学生1\第二步情感预测\svm_predictions.csv', index=False)
print(f"SVM预测结果已成功保存到 svm_predictions.csv")



Training and predicting for area:  军事


ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [8]:
# 获取BERT特征
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state
    return embeddings[:, 0, :].squeeze().numpy()

# 将情感词典权重与BERT特征结合的方法
def apply_sentiment_weights(bert_feature, text):
    vectorizer = CountVectorizer(vocabulary=tokenizer.get_vocab())
    word_counts = vectorizer.fit_transform([text]).toarray().flatten()
    words = vectorizer.get_feature_names_out()
    
    weighted_bert_feature = bert_feature.copy()
    for idx, word in enumerate(words):
        if word in sentiment_dict:
            sentiment_weight = sentiment_dict[word]
            weighted_bert_feature += sentiment_weight * word_counts[idx]
    
    return weighted_bert_feature

